# **Detect and Classify Tremor from the IRIS Database**
---
Workflow:
- Imports
- Inputs
- Define Functions
- Define Detect and Classify Function
- Bin by Days
- Apply Function & Process in Parallel 
- Concat Results & Save

## Imports

In [1]:
import datetime
from obspy.clients.fdsn.client import Client
import obspy
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
import dask
from dask.diagnostics import ProgressBar

#client = Client('IRIS')
import pickle
import os
import scipy.signal
import scipy.ndimage

from obspy.signal.trigger import classic_sta_lta
from obspy.signal.trigger import trigger_onset
from obspy import UTCDateTime
from obspy import Stream

from pnwstore.mseed import WaveformClient
from obspy import Stream
from datetime import timedelta


from obspy.clients.fdsn import Client
from datetime import timedelta, datetime

import logging
logging.basicConfig(level=logging.INFO)

print('done')

done


In [2]:
# gets data locally rather than online
client = WaveformClient()

## Inputs
Short Term Average & Long Term Average (STA/LTA) parameters

In [3]:
# Define step we want to chunk
step = 10 # seconds

# Define sta and lta window lengths
# Below chosen to optimize emergent signal detection!

sta_win = 10 # seconds, short term window
lta_win = 1000 # seconds, long term window

Time Endpoints and Station

In [4]:
t1 = obspy.UTCDateTime("2012-09-01T00:00:00.000")
t2 = obspy.UTCDateTime("2013-06-18T00:00:00.000")

network = '7D'
station = 'J10B' 
channel = 'BH2'
location = '*'

# Get response info for this station from IRIS
iris_client = Client('IRIS')
sta_response = iris_client.get_stations(network=network, station=station,starttime=t1,endtime=t2,level='response')
save_dir = 'classifications/'

## Define Functions
- Detect/Triggering: Check Data, Calculate STL/STA
- Classify: Pick Peaks, Apply Gaussian, Ship Noise
- Attach Response Cascadia
- Waveform across Midnight

Check Data

In [5]:
def check_data(stream,t1,sr,step,data_time_length):
    
    data = stream[0].data
    data_sample_length = sr * data_time_length

    
    # Catch for if data stream is less than specified
    if len(data) < data_sample_length:
        data_sample_length = int(len(data) - (len(data)%(sr*step)))
        data_time_length = int(data_sample_length / sr)
        
    # Catch for if start time is not as specified
    if stream[0].stats.starttime != t1:
        t1 = stream[0].stats.starttime
    
    return(t1,data_time_length)

Calculate Short Term Average & Long Term Average (STL/STA)

In [6]:
def calc_stalta(data,sr,step,data_time_length,sta_win,lta_win):
    
    # Matricize the data as step s chunks
    data_sample_length = sr * data_time_length
    
    
    chunked = np.reshape(data[0:data_sample_length],[int(data_sample_length/(sr*step)),int(sr*step)])
    chunked_medians = [np.median(chunked[i,:]) for i in range(np.shape(chunked)[0])]
    chunked_times = np.linspace(0,data_time_length,len(chunked_medians))
    
    # Step through and calculate sta & lta every step s
    sta = []
    lta = []
    for i,vec in enumerate(chunked_medians):

        # STA is median of the next window
        nwin = int(sta_win / step)
        sta.append(np.median(chunked_medians[i:i+nwin]))


        # LTA is median of the past window
        nwin = int(lta_win / step)
        lta.append(np.median(chunked_medians[i-nwin:i]))
    
    stalta = np.array(sta)/np.array(lta)
    
    return(stalta,chunked_times)

Pick Peaks

In [7]:
def pick_peaks_welch(trace,sampling_rate,nperseg_multiple,microseism_cutoff=True):
    """
    Estimate power spectra of a trace using Welch's method
    Pick peaks within the spectra!
    
    INPUTS:
    trace = obspy object, waveform
    sampling_rate = sampling rate of trace
    nperseg_multiple = length of each segment used to construct the Welch spectrum
    microseism_cutoff = Bool, whether or not to cut off the lower end of the spectrum to avoid the microseism
    
    OUTPUTS:
    f = frequencies of the spectra
    Pxx_den = associated power at each frequency, in decibels
    peak_ind = index of peaks within the spectra (f and Pxx_den), if found
    peaks = picked peak object from scipy
    median_power = median power of spectra from 20-80 Hz in decibels
    """
    
    
    fs = sampling_rate
    x = trace.data
    nperseg = fs * nperseg_multiple
    
    f,Pxx_den = scipy.signal.welch(x,fs,nperseg=nperseg)
    if microseism_cutoff is True:
        f = f[4:]
        Pxx_den = Pxx_den[4:]
        
    Pxx_den = [10*np.log10(d) for d in Pxx_den]
    median_power = np.median(Pxx_den[20:80])
    
    peaks = scipy.signal.find_peaks(Pxx_den,threshold =median_power*5,prominence=10) 
    peak_ind = peaks[0]
    
    return(f,Pxx_den,peak_ind,peaks,median_power)

Apply Gaussian

In [8]:
def apply_gaussian(filtered_data,samp_rate,gaussian_width=5):
    """
    Smooth waveform using a gaussian window
    
    INPUTS
    filtered_data = filtered numpy array of seismic data (from an obspy trace)
    samp_rate = sampling rate of data
    gaussian width = width of Gaussian window in seconds
    
    OUTPUTS
    smoothed_window = smoothed numpy array of seismic data
    """
    
    # Square data
    data = filtered_data**2
    
    gaussian_radius = int((gaussian_width * samp_rate)/2)
    smoothed_window=scipy.ndimage.gaussian_filter1d(data,sigma=gaussian_radius/4,radius=gaussian_radius)
    
    return smoothed_window

Ship Noise

In [9]:
def ship_noise_classifier(trace,sampling_rate):
    """
    Check whether detection likely includes ship noise in the form of a spectral peak
    
    INPUTS
    trace = obspy trace object
    sampling_rate = sample rate of trace
    
    OUTPUTS
    ship_classifier = number of peaks in the spectra. If any exist, ship noise is likely!
    """
    
    
    # Pick peaks on the smoothed spectrum of the trace (nperseg multiple = 1)
    f,Pxx_den,peak_ind,peak_details,median_power = pick_peaks_welch(trace,sampling_rate,1,microseism_cutoff=True)
    
    if len(peak_ind)==0:
        ship_classifier = 0
    else:
        ship_classifier = len(peak_ind)

    
    return ship_classifier

In [10]:
# NOTE: SHOULDN'T NEED THIS ANYMORE
# PULL UP-TO-DATE STATION INFO FROM IRIS AT TOP OF NOTEBOOK

# def attach_response_cascadia(stream):
#     """
#     Reads in an obspy stream
#     Pulls down station response information for the corresponding station from where it is stored in Cascadia
#     Attaches this station response information to the stream, and returns the stream
#     """
    
#     # This is where station response information is stored on Cascadia, in xml format:
#     dir_base = '/1-fnp/pnwstore1/p-wd11/PNWStationXML/'
    
#     network = stream[0].stats.network
#     station = stream[0].stats.station
    
#     # Response file paths are in the format /1-fnp/pnwstore1/p-wd11/PNWStationXML/OO/OO.HYSB1.xml
#     response_file = dir_base+network+'/'+network+'.'+station+'.xml'

#     # Read response info using obspy
#     inventory = obspy.read_inventory(response_file)

#     # Attach response information to stream
#     stream.attach_response(inventory)
    
#     return stream

In [11]:
def get_waveform_across_midnight(client, network, station, location, channel, starttime, endtime):
    stream = Stream()
    current = starttime
    while current < endtime:
        end_of_day = obspy.UTCDateTime(current.date) + 86399  # 23:59:59
        chunk_end = min(end_of_day, endtime)
        #print(current, chunk_end)
        #try:
        st_chunk = client.get_waveforms(
            network=network, station=station,
            location=location, channel=channel,
            starttime=current, endtime=chunk_end,
        )
        sr = round(st_chunk[0].stats.sampling_rate)
        st_chunk.resample(sr).merge(fill_value="interpolate")
        stream += st_chunk
        #except Exception as e:
        #    print(f"Failed to get data from {current} to {chunk_end}: {e}")
        current = obspy.UTCDateTime(current.date + timedelta(days=1))
    return stream

## Define Detect and Classify Function

In [12]:
def trigger_and_classify(t1, data_time_length, network, station, channel, sta_response, step, sta_win, lta_win, filepath):
    """
    Detects and classifies seismic events using STA/LTA and signal features.
    """
    
    # Adjust times with padding
    t1 = obspy.UTCDateTime(t1)
    t2 = t1 + data_time_length + 1000
    t1 = t1 - 1000

    try:
        client = WaveformClient() 

        # === 1. Detection ===
        st1 = get_waveform_across_midnight(client, network, station, "*", channel, t1, t2)
        st1.merge(fill_value= "interpolate")
        #print(f"Stream length: {len(st1)}")
        if len(st1) == 0:
            print(f"[{t1}] No traces found in stream. Skipping.")
            return [], []

        sr = round(st1[0].stats.sampling_rate)
        if st1[0].stats.sampling_rate != sr:
            st1.resample(sr).merge(fill_value="interpolate")
        st1.taper(0.05, max_length=5)
        st1.filter("bandpass", freqmin=3, freqmax=10)
        st1.trim(starttime=t1, endtime=t2)

        t1, data_time_length = check_data(st1, t1, sr, step, data_time_length)
        data = np.abs(st1[0].data)
        #print(f"Analyzing {st1[0].stats.station} {st1[0].stats.channel}")
        stalta, times = calc_stalta(data, sr, step, data_time_length, sta_win, lta_win)
        triggers = trigger_onset(stalta, 2, 1)
        ontimes = [t1 + times[tr[0]] for tr in triggers]
        offtimes = [t1 + times[tr[1]] for tr in triggers] 
        # Get rid of any detections less than 30 s
        keep_ind = []
        for j in range(len(ontimes)):
            if offtimes[j]-ontimes[j] > 30:
                keep_ind.append(j)
        ontimes = [ontimes[j] for j in keep_ind]
        offtimes = [offtimes[j] for j in keep_ind]
        # print(ontimes,offtimes)

        # === 2. Classification ===
        all_results = []

        for on, off in zip(ontimes, offtimes):
            #try:
            t1c, t2c = on, off

            # Ship noise classifier
            st_ship = get_waveform_across_midnight(client, network, station, "*", channel, t1c-5, t2c+5)
            st_ship.resample(sr).merge(fill_value="interpolate")
            # st_ship = attach_response_cascadia(st_ship)
            st_ship.attach_response(sta_response)
            st_ship[0].data = st_ship[0].data / st_ship[0].stats.response.instrument_sensitivity.value
            st_ship.trim(starttime=t1c, endtime=t2c)
            ship_classifier = ship_noise_classifier(st_ship[0], sr)

            # Waveform peak classifier
            st_wave = get_waveform_across_midnight(client, network, station, "*", channel, t1c-5, t2c+5)
            st_wave.resample(sr).merge(fill_value="interpolate")
            # st_wave = attach_response_cascadia(st_wave)
            st_wave.attach_response(sta_response)
            st_wave.filter("bandpass", freqmin=3, freqmax=10)
            st_wave.remove_response()
            st_wave.trim(starttime=t1c, endtime=t2c)
            max_amplitude = np.max(np.abs(st_wave[0].data))
            smoothed = apply_gaussian(st_wave[0].data, sr, gaussian_width=15)
            smoothed = smoothed / np.max(smoothed)
            peaks = scipy.signal.find_peaks(smoothed, prominence=0.1)
            num_waveform_peaks = len(peaks[0])

            # Welch frequency ratio
            t1w = t1c - 30
            t2w = t2c + 30
            st_welch = get_waveform_across_midnight(client, network, station, "*", channel, t1w-5, t2w+5)
            st_welch.resample(sr).merge(fill_value="interpolate")
            # st_welch = attach_response_cascadia(st_welch)
            st_welch.attach_response(sta_response)
            st_welch[0].data = st_welch[0].data / st_welch[0].stats.response.instrument_sensitivity.value
            st_welch.trim(starttime=t1w, endtime=t2w)
            f, Pxx_den, *_ = pick_peaks_welch(st_welch[0], sr, 5, microseism_cutoff=False)
            freq_ratio_welch = 10 ** (np.median(Pxx_den[25:50]) / 10) / 10 ** (np.median(Pxx_den[50:75]) / 10)

            # Save result
            all_results.append([
                (t1c, t2c), station, num_waveform_peaks,
                ship_classifier, freq_ratio_welch, max_amplitude
            ])

            #except Exception as e:
                #print(f"Classification failed for detection starting at {str(on)}: {e}")
                #continue

        # === Save all results ===
        # combined_file = filepath + station + "_all_detections.pickle"
        # with open(combined_file, "wb") as handle:
        #     pickle.dump(all_results, handle)

        return all_results

    except Exception as e:
        print(f"[{str(t1)}] No data or fatal error during detection/classification: {e}")
        return [], []


## Bin data by days

In [13]:
time_bins = pd.date_range(start=t1.datetime, end=t2.datetime, freq='D')
data_time_length = 24 * 60 * 60
pickle_path = 'classifications/J10B_raw.pickle'

In [ ]:
@dask.delayed
def loop_days(t, data_time_length, network, station, channel, sta_reseponse, step, sta_win, lta_win, filepath):
    t_utc = obspy.UTCDateTime(t)
    return trigger_and_classify(t_utc, data_time_length, network, station, channel, sta_response, step, sta_win, lta_win, filepath)

# Build lazy results list
lazy_results = [
    loop_days(t, data_time_length, network, station, channel, sta_response, step, sta_win, lta_win, save_dir)
    for t in time_bins
]

# Run Dask
with ProgressBar():
    results = dask.compute(*lazy_results)

[                                        ] | 0% Completed | 920.26 ms[2013-04-15T23:43:20.000000Z] No data or fatal error during detection/classification: list index out of range
[                                        ] | 0% Completed | 1.03 s[2013-02-08T23:43:20.000000Z] No data or fatal error during detection/classification: list index out of range
[                                        ] | 0% Completed | 1.23 s[2013-03-09T23:43:20.000000Z] No data or fatal error during detection/classification: list index out of range
[                                        ] | 1% Completed | 1.34 s[2012-11-06T23:43:20.000000Z] No data or fatal error during detection/classification: list index out of range
[                                        ] | 1% Completed | 2.77 s

/home/jupyter_share/miniconda3/envs/seismo2/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/jupyter_share/miniconda3/envs/seismo2/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


[                                        ] | 1% Completed | 2.97 s[2013-04-14T23:43:20.000000Z] No data or fatal error during detection/classification: list index out of range
[                                        ] | 1% Completed | 4.03 s

[                                        ] | 1% Completed | 5.09 s

[                                        ] | 1% Completed | 6.36 s

...

[###################################     ] | 89% Completed | 540.38 s

[###################################     ] | 89% Completed | 540.83 s

[###################################     ] | 89% Completed | 541.27 s

[####################################    ] | 90% Completed | 541.83 s

[####################################    ] | 90% Completed | 542.05 s

## Concat Results & Save

In [ ]:
os.chdir('/home/lydiagk/obs_tremor/tremor_detection')
print("Current working directory:", os.getcwd())

In [ ]:
# removing empty brackets
cleaned_results = [entry for entry in results if entry]
with open(pickle_path, 'wb') as f:
    pickle.dump(cleaned_results, f)

print(f"Saved combined detection results to {pickle_path}")
print(f"Total detections: {len(cleaned_results)}")


In [ ]:
with open(pickle_path, 'rb') as f:
    raw_triggers = pickle.load(f)

flattened = [
    detection
    for day in raw_triggers
    if isinstance(day, list)
    for detection in day
    if detection and isinstance(detection, (list, tuple)) and len(detection) == 6]

with open('classifications/J10B.pickle', 'wb') as f:
    pickle.dump(flattened, f)


## Checking my work against validated code

In [ ]:
os.chdir("/home/lydiagk/obs_tremor/tremor_detection")


start_time = UTCDateTime("2017-09-01T00:00:00.000")
end_time = UTCDateTime("2017-09-08T00:00:00.000")


with open('flat_triggers.pickle', 'rb') as f:
    my_triggers = pickle.load(f)
   
count = sum(1 for entry in my_triggers if start_time <= entry[0][0] <= end_time)

print(f"Number of triggers between {start_time} and {end_time}: {count}")
starts = [entry[0][0] for entry in my_triggers if start_time <= entry[0][0] <= end_time]
peaks = [entry[2] for entry in my_triggers if start_time <= entry[0][0] <= end_time and entry[2] == 1]
print(len(peaks))
print(peaks)


In [ ]:
os.chdir("/home/lydiagk/obs_tremor/tremor_detection/results")

# define time window
start_time_v = UTCDateTime("2017-09-01T00:00:00.000")
end_time_v = UTCDateTime("2017-09-08T00:00:00.000")

with open('HYSB1_HHN_3-10Hz_classifications.pickle', 'rb') as f:
    my_triggers_v = pickle.load(f)


count_v = sum(1 for entry in my_triggers_v if start_time_v <= entry[0][0] <= end_time_v)

print(f"Number of triggers between {start_time_v} and {end_time_v}: {count_v}")
starts_v = [entry[0][0] for entry in my_triggers_v if start_time_v <= entry[0][0] <= end_time_v]
peaks_v = [entry[2] for entry in my_triggers_v if start_time <= entry[0][0] <= end_time and entry[2] == 1]

print(len(peaks_v))
print(peaks_v)